<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example_covariate_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install git+https://github.com/yeoun9/torchpm.git
! pip install torchpm

In [2]:
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.parameter import *
from torchpm import covariate
import torch as tc
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.stats import zmap
import pickle
import os

In [3]:
def run(seed):
    np.set_printoptions(threshold=sys.maxsize)
    np.random.seed(seed)
    dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO.csv'
    dataset_np = np.loadtxt(dataset_file_path, delimiter=',', dtype=np.float32, skiprows=1)
    column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', "MDV", "RATE", 'BWT_Norm']

    ids, ids_start_idx = np.unique(dataset_np[:, column_names.index('ID')], return_index=True)


    def set_data(dataset_np, ids_start_idx, data_for_each_id, comlumn_name, column_names) :
        column_index = column_names.index(comlumn_name)
        for i, start_idx in enumerate(ids_start_idx):
            
            if i >= len(ids_start_idx) - 2 : 
                indice = slice(start_idx,record_length)
            else : 
                indice = slice(start_idx,ids_start_idx[i+1])    
            dataset_np[indice, column_index] = data_for_each_id[i]
        return dataset_np


    record_length = len(dataset_np)
    dataset_np_splited_by_id = np.split(dataset_np, ids_start_idx[1:])


    bwt_index = column_names.index('BWT_Norm')
    bwt_each_id = []
    for data_by_id in dataset_np_splited_by_id :
        bwt_each_id.append(data_by_id[0,bwt_index])
    bwt_each_id = zmap(bwt_each_id, bwt_each_id)

    dataset_np = set_data(dataset_np, ids_start_idx, bwt_each_id, 'BWT_Norm', column_names)


    column_names.append('fixed')
    fixed = np.zeros([record_length,1], dtype=np.float32)
    dataset_np = np.hstack([dataset_np,fixed])


    column_names.append('rand-1+1')
    fixed = np.zeros([record_length,1], dtype=np.float32)
    dataset_np = np.hstack([dataset_np,fixed])
    random_for_id = (np.random.rand(12) - 0.5)*2
    dataset_np = set_data(dataset_np, ids_start_idx, random_for_id, 'rand-1+1', column_names)

    column_names.append('norm(0,1)')
    fixed = np.zeros([record_length,1], dtype=np.float32)
    dataset_np = np.hstack([dataset_np,fixed])
    norm_for_id = np.random.normal(0, 1, 12)
    dataset_np = set_data(dataset_np, ids_start_idx, norm_for_id, 'norm(0,1)', column_names)


    def add_cor(cor_value, dataset_np) :
        fixed = np.zeros([record_length,1], dtype=np.float32)
        dataset_np = np.hstack([dataset_np,fixed])
        cor_value = cor_value
        column_names.append('BWT_Cor_'+str(cor_value))
        BWT_Cor_for_id = cor_value*bwt_each_id + np.random.normal(0, 1, 12) * np.sqrt(1-cor_value**2)
        dataset_np = set_data(dataset_np, ids_start_idx, BWT_Cor_for_id, 'BWT_Cor_'+str(cor_value), column_names)
        return dataset_np

    for i in [1, 5]:
        dataset_np = add_cor(i/10, dataset_np)

    device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
    dataset = CSVDataset(dataset_np, column_names, device)

    class BasementModel(predfunction.PredictionFunctionByTime) :
        
        def _set_estimated_parameters(self):
            self.theta_0 = Theta(0., 1.5, 10.)
            self.theta_1 = Theta(0., 30., 100.)
            self.theta_2 = Theta(0, 0.08, 1)

            self.eta_0 = Eta()
            self.eta_1 = Eta()
            self.eta_2 = Eta()

            self.eps_0 = Eps()
            self.eps_1 = Eps()

            self.gut_model = linearode.Comp1GutModelFunction()
        
        def _calculate_parameters(self, covariates):
            covariates['k_a'] = self.theta_0()*tc.exp(self.eta_0())
            covariates['v'] = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
            covariates['k_e'] = self.theta_2()*tc.exp(self.eta_2())
            covariates['AMT'] = tc.tensor(320., device=self.dataset.device)

        def _calculate_preds(self, t, p):
            dose = p['AMT'][0]
            k_a = p['k_a']
            v = p['v']
            k_e = p['k_e']
            comps = self.gut_model(t, k_a, k_e, dose)
            return comps[1]/v
            
        def _calculate_error(self, y_pred, p):
            p['v_v'] = p['v'] 
            return y_pred +  y_pred * self.eps_0() + self.eps_1()

    dependent_parameter_names = ['k_a', 'v', 'k_e']
    dependent_parameter_initial_values = [[0,1.4901,2],[30,32.4667,34],[0,0.08,0.1]]
    independent_parameter_names = column_names[column_names.index('BWT_Norm'):]

    searcher = covariate.DeepCovariateSearching(dataset=dataset,
                                    BaseModel=BasementModel,
                                    dependent_parameter_names=dependent_parameter_names,
                                    independent_parameter_names=independent_parameter_names,
                                    dependent_parameter_initial_values=dependent_parameter_initial_values,
                                    eps_names=['eps_0', 'eps_1'],
                                    omega = Omega([0.4397,
                                                    0.0575,  0.0198, 
                                                    -0.0069,  0.0116,  0.0205], False, requires_grads=True),
                                    sigma = Sigma([[0.0177], [0.0762]], [True, True], requires_grads=[True, True]))
    result = searcher.run(tolerance_grad=1e-3, tolerance_change=1e-3)
    return result

In [4]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder('covariate_searching_result')

for seed in range(0,200):
    try :
        file_list = os.listdir('covariate_searching_result')
        result_file_list = [k for k in file_list if 'seed' in k]
        if len(result_file_list) > 99 : break
        
        result = run(seed)
        with open('covariate_searching_result/seed' + str(seed) +'.pkl', 'wb') as fp:
            pickle.dump(result, fp)
    except :
        print('error')



In [32]:
cov_dict = {}
count = 0
for i in range(130):
    try :
    
        with open('covariate_searching_result/seed' + str(i) + '.pkl', 'rb') as fp:
            data = pickle.load(fp)
        for cov_name in data['selected covariates']:
            if cov_name not in cov_dict.keys() :
                cov_dict[cov_name] = 1
            else :
                cov_dict[cov_name] += 1
        count += 1
    except:
        pass
print(count)
print("counts for selected covariate in", count, "times covariate searching")
sorted_dict = sorted(cov_dict.items(), key = lambda item: item[1], reverse = True)
print(sorted_dict)
with open('covariate_searching_result/seed' + str(4) + '.pkl', 'rb') as fp:
    data = pickle.load(fp)
print(data['selected covariates'])

for i, record in enumerate(data['history']) :
    data['history'][i]['>3.84'] = str(record['loss difference'] > 3.84)
    data['history'][i]['selected'] = 'selected' if record['loss difference'] > 3.84 else 'removed'

for i, record in enumerate(data['history']) :
    if i > 0 :
        if data['history'][i-1]['selected'] == 'selected' :
            record['selected'] = ''
            record['>3.84'] = ""
    print(record)

100
counts for selected covariate in 100 times covariate searching
[('BWT_Norm', 82), ('BWT_Cor_0.5', 71), ('BWT_Cor_0.1', 63), ('rand-1+1', 62), ('norm(0,1)', 55)]
['BWT_Norm', 'norm(0,1)', 'BWT_Cor_0.5']
{'loss': 69.52711486816406, 'removed covariates': [], 'loss difference': 0.0, '>3.84': 'False', 'selected': 'removed'}
{'loss': 80.7589340209961, 'removed covariates': ['BWT_Norm'], 'loss difference': 11.231819152832031, '>3.84': 'True', 'selected': 'selected'}
{'loss': 69.52711486816406, 'removed covariates': [], 'loss difference': -11.231819152832031, '>3.84': '', 'selected': ''}
{'loss': 69.49146270751953, 'removed covariates': ['fixed'], 'loss difference': -0.03565216064453125, '>3.84': 'False', 'selected': 'removed'}
{'loss': 72.25386047363281, 'removed covariates': ['fixed', 'rand-1+1'], 'loss difference': 2.7623977661132812, '>3.84': 'False', 'selected': 'removed'}
{'loss': 77.00479125976562, 'removed covariates': ['fixed', 'rand-1+1', 'norm(0,1)'], 'loss difference': 4.750930